# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355109040660                   -1.09    6.0    161ms
  2   -9.356807663297       -2.77       -1.44    1.0   82.6ms
  3   -9.357077909041       -3.57       -2.75    2.2    103ms
  4   -9.357119558651       -4.38       -3.00    6.4    513ms
  5   -9.357119980686       -6.37       -3.16    2.8    110ms
  6   -9.357120158738       -6.75       -3.31    1.0    106ms
  7   -9.357120259943       -6.99       -3.47    1.0   80.4ms
  8   -9.357120300242       -7.39       -3.63    1.0   79.8ms
  9   -9.357120318628       -7.74       -3.90    4.5    130ms
 10   -9.357120322682       -8.39       -4.16    1.0   82.4ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74766758620                   -0.97    6.5    517ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     6
│     1
│     7
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.75919344035       -1.94       -1.32    2.4    289ms
  3   -18.79221508166       -1.48       -2.18    4.0    341ms
  4   -18.79261146728       -3.40       -2.51    6.6    502ms
  5   -18.79262136677       -5.00       -3.21    1.5    260ms
  6   -18.79262398715       -5.58       -3.61    5.9    481ms
  7   -18.79262416531       -6.75       -3.97    1.5    238ms
  8   -18.79262417690       -7.94       -4.50    2.9    323ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54961905059                   -0.84    9.5    1.83s
  2   -37.55770475831       -2.09       -1.22    2.6    778ms
  3   -37.56017795822       -2.61       -2.26    6.1    1.14s
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     13
│      6
│     16
│     23
│     25
│     25
│     17
│     10
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  4   -37.56489247538       -2.33       -2.24   16.9    2.18s
  5   -37.56493031763       -4.42       -2.32    3.0    908ms
  6   -37.56498330887       -4.28       -3.42    1.0    811ms
  7   -37.56498400694       -6.16       -3.44   11.4    2.06s
  8   -37.56498501644       -6.00       -3.79    1.0    766ms
  9   -37.56498516867       -6.82       -4.38    3.0    1.30s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355141650162                   -1.10    6.1    184ms
  2   -9.356838975310       -2.77       -1.91    1.0   74.9ms
  3   -9.357106551049       -3.57       -2.77    4.1    133ms
  4   -9.357099462590   +   -5.15       -2.60    6.2    174ms
  5   -9.357119952128       -4.69       -3.50    1.0   84.8ms
  6   -9.357120265798       -6.50       -3.95    2.0   95.9ms
  7   -9.357120321419       -7.25       -4.50    3.5    123ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55003831952                   -0.84   10.2    2.08s
  2   -37.54019317651   +   -2.01       -1.57    2.8    694ms
  3   -20.54240602366   +    1.23       -0.45    9.9    1.81s
  4   -37.52566122276        1.23       -1.72   10.1    1.82s
  5   -37.54939045633       -1.62       -1.88    2.0    1.07s
  6   -37.23552607710   +   -0.50       -1.30    6.0    1.24s
  7   -37.49702516436       -0.58       -1.58    6.1    1.22s
  8   -37.56403888607       -1.17       -2.24    3.4    958ms
  9   -37.56415021508       -3.95       -2.35    2.9    946ms
 10   -37.56454016611       -3.41       -2.44    2.2    1.13s
 11   -37.56490838851       -3.43       -2.93    1.8    798ms
 12   -37.56497516424       -4.18       -3.32    2.6    1.03s
 13   -37.56498131626       -5.21       -3.40    3.0    893ms
 14   -37.56498400392       -5.57       -3.67    1.2    686ms
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.